In [ ]:
import torch

from attention_approximation.modeling_llama_approximated import LlamaModel as StudentModel
from attention_approximation.modeling_llama import LlamaForCausalLM as TeacherModel
from transformers import LlamaConfig
from attention_approximation.pytorch import intersect_dicts
from attention_approximation.utils import LOGGER
import safetensors
import copy

config = "../data/MobileLLM/config.json"
w1 = "../data/MobileLLM/model.safetensors"
w2 = "/Users/gianlucacalo/Desktop/checkpoints/checkpoint_step_1001.pt"

config = LlamaConfig().from_json_file(config)
teacher = TeacherModel(config)
ckpt = safetensors.torch.load_file(w1)
csd = intersect_dicts(ckpt, teacher.state_dict())
teacher.load_state_dict(csd)
LOGGER.info(f"Transferred {len(csd)}/{len(teacher.state_dict())} items from pretrained weights")


config = copy.deepcopy(config)
config.factorization_rank = 16
config.layer_sharing = False
config.seq_length = 512

student = StudentModel(config)
ckpt = torch.load(w2, map_location="cpu")['model_state_dict']


csd = intersect_dicts(ckpt, student.state_dict())
student.load_state_dict(csd)
LOGGER.info(f"Transferred {len(csd)}/{len(student.state_dict())} items from pretrained weights")




LlamaForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Transferred 272/272 items from pretrained weights
Transferred 212/212 items from pretrained weights
